# Data Preperation

### Importiere Bibliotheken

In [104]:
import pandas as pd
from imblearn.combine import SMOTETomek
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

### Datensatz einlesen
Da Pandas die Datentypen der Merkmale nicht optimal erkennt, werden im Folgenden die konkreten Datentypen definiert. \
Diese können aus der Beschreibung des Datensatztes abgeleitet werden.

In [105]:
dtypes = {
    'UDI': 'int32',
    'Product ID': 'str',
    'Type': 'category',
    'Air temperature [K]': 'float32',
    'Process temperature [K]': 'float32',
    'Rotational speed [rpm]': 'float32',
    'Torque [Nm]': 'float32',
    'Tool wear [min]': 'float32',
    'Machine failure': 'bool',
    'TWF': 'bool',
    'HDF': 'bool',
    'PWF': 'bool',
    'OSF': 'bool',
    'RNF': 'bool'
}

df = pd.read_csv('./dataset.csv', dtype=dtypes)

### Erstelle Dummy-Merkmale für das Merkmal Maschinen-Typ

In [106]:
dummy_columns = pd.get_dummies(df['Type'], prefix='Type')
df = pd.concat([df, dummy_columns], axis=1)

### Definiere die Input- und Output- Merkmale

In [107]:
input_columns = ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]', 'Type_H', 'Type_L', 'Type_M']
output_column = ['label']
numeric_columns = ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']

### Generiere Labeling

In [108]:
def get_label(row):
    if row['Machine failure'] == 0:
        return 'no_failure'
    else:
        for defect in ['TWF', 'HDF', 'PWF', 'OSF', 'RNF']:
            if row[defect] == 1:
                return defect
    return 'unknown'

df['label'] = df.apply(get_label, axis=1)

### Initialisiere Training- und Testdaten

In [109]:
X = df[input_columns]
y = df[output_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

### Implementiere Oversampling auf den Trainingsdaten
Oversampling wird gewählt, da der Datensatz zwei sehr starke Unausgeglichenheiten enthält und ein Undersampling zu einem sehr starken Datenverlust führen würde.\
Dies liegt daran, da nur wenige Machinen tatsächlich einen Defekt aufweisen.

In [110]:
smote = SMOTETomek(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

### Kombiniere Test- und Trainingsdatensatz

In [111]:
df_train_resampled = pd.concat([X_train_resampled, y_train_resampled], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

### Skaliere numerische Werte mit der Standartskalierung auf Basis der Trainingsdaten

In [112]:
scaler = StandardScaler()
df_train_resampled[numeric_columns] = scaler.fit_transform(df_train_resampled[numeric_columns])
df_test[numeric_columns] = scaler.transform(df_test[numeric_columns])

### Speichere den Trainings- und Testdatensatz

In [113]:
df_train_resampled.to_csv('./dataset_train_resampled.csv')
df_test.to_csv('./dataset_test.csv')

In [114]:
pd.concat([df_train_resampled, df_test]).describe(include='all')

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Type_H,Type_L,Type_M,label
count,48312.000000,48312.000000,48312.000000,48312.000000,48312.000000,48312,48312,48312,48312
unique,NaN,NaN,NaN,NaN,NaN,2,2,2,6
top,NaN,NaN,NaN,NaN,NaN,False,True,False,no_failure
freq,NaN,NaN,NaN,NaN,NaN,40806,38246,27134,9630
mean,-0.010502,-0.003960,0.001772,-0.016196,-0.022323,NaN,NaN,NaN,NaN
std,1.004571,1.009924,0.984980,0.991435,1.004071,NaN,NaN,NaN,NaN
min,-2.794007,-3.638179,-1.080400,-2.850194,-2.138733,NaN,NaN,NaN,NaN
25%,-0.827600,-0.702615,-0.498136,-0.626878,-0.844873,NaN,NaN,NaN,NaN
50%,0.006652,0.101700,-0.313453,0.117214,0.181291,NaN,NaN,NaN,NaN
75%,0.884879,0.696146,0.082452,0.673344,0.921724,NaN,NaN,NaN,NaN
